<a href="https://colab.research.google.com/github/k-dinakaran/automation-of-wordpress-post-publication-using-AI-tools/blob/main/develop_AI_tool_for_summarizing_user_comments_on_generated_contents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas nltk transformers


In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

nltk.download('punkt')

# Function to preprocess text
def preprocess_text(text):
    # Remove non-alphabet characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenize into sentences
    sentences = sent_tokenize(text)
    # Remove stop words from sentences
    cleaned_sentences = []
    for sentence in sentences:
        cleaned_sentence = ' '.join([word for word in sentence.split() if word not in ENGLISH_STOP_WORDS])
        cleaned_sentences.append(cleaned_sentence)
    return ' '.join(cleaned_sentences)  # Join back the cleaned sentences

# Function to summarize text
def summarize_text(text):
    max_chunk = 500  # Define max chunk size for the model
    sentences = text.split('. ')  # Split text into sentences
    current_chunk = 0
    chunks = []

    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            chunks.append(sentence.split(' '))

    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    res = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
        res.append(summary[0]['summary_text'])

    # Post-process to remove any exact repetitions in the summary
    final_summary = ' '.join(res)
    sentences = sent_tokenize(final_summary)
    unique_sentences = list(dict.fromkeys(sentences))  # Remove duplicates
    return ' '.join(unique_sentences)

# Get user input for file path and content ID
file_path = '/content/summarize comment.csv'  # Update this to the actual path where your file is located
content_id = input("Enter the content ID: ")

# Load the dataset
data = pd.read_csv(file_path)

# Define column names based on the provided dataset structure
content_id_column = 'Content ID'
comment_column = 'User_Comment'

# Filter the dataset for the specified content ID
filtered_data = data[data[content_id_column] == content_id]

# Check if any comments are found for the specified content ID
if filtered_data.empty:
    print(f"No comments found for content ID {content_id}")
else:
    # Preprocess comments
    filtered_data['cleaned_comments'] = filtered_data[comment_column].apply(preprocess_text)

    # Initialize summarization pipeline
    summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

    # Concatenate all cleaned comments into a single string
    all_comments = ' '.join(filtered_data['cleaned_comments'])

    # Summarize the concatenated comments
    summary = summarize_text(all_comments)

    # Display the summary
    print(f"\n{'-'*70}")
print(f"Summary of comments for content ID {content_id}:")
print(f"{summary}")
print(f"{'-'*70}\n")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Enter the content ID: A01


<ipython-input-2-165c957ae587>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['cleaned_comments'] = filtered_data[comment_column].apply(preprocess_text)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Your max_length is set to 200, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)



----------------------------------------------------------------------
Summary of comments for content ID A01:
 excellent article investment informative. excellent article. investment informative on how to get the most out of your investment. great article on how not to lose money on your investments. fantastic article on why not to invest in your own company's stock. excellent investment.
----------------------------------------------------------------------

